In [ ]:
import script
from flask import Flask, jsonify, request
import pandas as pd
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import json


In [19]:
app = Flask(__name__)

In [20]:
@app.route('/api/recherche_nom', methods = ['GET'])
def get_recherche_nom():
    args = request.args
    name = args.get("name")
    df = pd.read_csv("../src/recipesDB/Small_Recipes.csv")
    df1 = df[["name","id"]].copy()
    df1 = df1.set_index("id")
    df1 = df1["name"].str.contains(name) 
    result = df1.loc[df1 == True].to_frame().index.to_list()
    df2 = []
    for i in result:
        recipe = script.SmallRecipe(int(i))
        df2.append(recipe.__dict__())
    return {'recipes': df2}, 200

In [21]:
@app.route('/api/recette', methods = ['GET'])
def get_recette_all():
    args = request.args
    id = args.get("id")
    if id is None:
        df = pd.read_csv("../src/recipesDB/Small_Recipes.csv")
        id = list(df['id'])
        df1 = []
        for i in id:
            recipe = script.SmallRecipe(int(i))
            df1.append(recipe.__dict__())
        return {'recipes': df1}, 200
    recipe = script.SmallRecipe(int(id))
    return {'recipes': recipe.__dict__()}, 200

In [22]:
@app.route('/api/machine_learning', methods = ['GET'])
def get_machine_learning():
    args = request.args
    id = args.get("id")
    recipe_list = script.main_function(int(id))
    df1 = []
    for i in recipe_list:
        response = requests.get('http://www.food.com/recipe/' + str(i))
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            suggestion = script.MediumRecipe(int(i), soup)
            df1.append(suggestion.__dict__())
    return {'suggestion': df1}, 200


In [23]:
@app.route('/api/utilisateur', methods = ['POST'])
def post_nouvel_utilisateur():
    df = pd.read_csv("../src/recipesDB/User.csv")
    args = request.args
    firstName = args.get("FirstName")
    lastName = args.get("LastName")
    password = args.get("Password")
    mail = args.get("Email")
    avatar = args.get("MyAvatar")
    date = datetime.today().strftime('%Y-%m-%d')
    id = df.last().id + 1
    likes = []
    user = pd.DataFrame({"id":[id],"firstName": [firstName], "lastName": [lastName], "password": [password], "mail": [mail], "avatar": [avatar], "date": [date]})
    user.to_json("../src/recipesDB/User.csv", mode='a', header=False)

In [24]:
print(datetime.today().strftime('%Y-%m-%d'))

2022-07-05


In [25]:
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
